In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# Enter foldername
FOLDERNAME = 'cs231n/DeepDoughnut'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Ensures Python interpreter of Colab VM can load python files.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

#Set your path
path='/content/drive/Shared drives/DeepDoughnut'


In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import os
from scipy.io import loadmat

In [ ]:
mat = loadmat(os.path.join(path, 'tabular_input_data/GPS_Long_Lat_Compass.mat'))
newData = list(zip(mat['GPS_Compass'][:, 0], mat['GPS_Compass'][:, 1], mat['GPS_Compass'][:, 2]))
columns = ['lat', 'lon', 'compass']
df = pd.DataFrame(newData, columns=columns)
df.to_csv(os.path.join(path, 'output_data/ucf_lat_lon.csv'))

In [ ]:
df['merge_col']=df.index+1

In [ ]:
#####
latlonzip=pd.read_csv(os.path.join(path, 'tabular_input_data/zcta_gaz.txt'), sep='\t').iloc[:, [0,7,8]]
latlonzip.columns=['zip', 'lat', 'lon']

In [ ]:
def find_closest_zip(row, latlonzip):
  lat=row['lat']; lon=row['lon'];
  latlonzip['dist']=np.power(latlonzip['lat']-lat, 2) + np.power(latlonzip['lon']-lon, 2)
  return latlonzip.loc[latlonzip['dist'].idxmin(), 'zip']

In [ ]:
df['zip'] = df.apply(find_closest_zip, axis=1, latlonzip=latlonzip)

In [ ]:
df.to_csv(os.path.join(path, 'output_data/ucf_lat_lon_geocoded.csv'))

In [ ]:
## Read in zip code level outcome data
zillow=pd.read_csv('https://raw.githubusercontent.com/arjunramani3/donut-effect/main/data/zhvi_panel_zips.csv')\
  [['zip', 'post_pct_change', 'density2019', 'wfh_emp', 'dist_to_cbd', 'estab_count']]

In [ ]:
df=df.merge(zillow, on='zip', how='left')

In [ ]:
df.to_csv(os.path.join(path, 'output_data/ucf_lat_lon_geocoded_with_outcomes.csv'))

In [ ]:
df.head()

,lat,lon,compass,merge_col,zip,post_pct_change,density2019,wfh_emp,dist_to_cbd,estab_count
0,40.440309,-80.0000,115.74,1,15222,8.064086,6583.614873,0.484400,1016.711270,3021.0
1,40.440271,-80.0068,119.24,2,15211,18.814366,6657.455182,0.401119,1668.319496,175.0
2,40.440229,-80.0067,118.96,3,15211,18.814366,6657.455182,0.401119,1668.319496,175.0
3,40.440189,-80.0066,118.68,4,15211,18.814366,6657.455182,0.401119,1668.319496,175.0
4,40.440146,-80.0065,118.40,5,15211,18.814366,6657.455182,0.401119,1668.319496,175.0


In [ ]:
grid=zillow.merge(latlonzip, on='zip')[['lat', 'lon', 'post_pct_change']]

In [ ]:
import scipy

In [ ]:
rbf = scipy.interpolate.Rbf(grid['lat'], grid['lon'], grid['post_pct_change'], function='linear')

In [ ]:
pred = rbf(df['lat'], df['lon'])

In [ ]:
df['pred_post_pct_change']=pred

In [ ]:
df.to_csv(os.path.join(path, 'output_data/ucf_lat_lon_interpolated.csv'))

Regression

In [ ]:
df.head()

,lat,lon,compass,merge_col,zip,post_pct_change,density2019,wfh_emp,dist_to_cbd,estab_count,pred_post_pct_change
0,40.440309,-80.0000,115.74,1,15222,8.064086,6583.614873,0.484400,1016.711270,3021.0,12.631622
1,40.440271,-80.0068,119.24,2,15211,18.814366,6657.455182,0.401119,1668.319496,175.0,15.016420
2,40.440229,-80.0067,118.96,3,15211,18.814366,6657.455182,0.401119,1668.319496,175.0,14.988317
3,40.440189,-80.0066,118.68,4,15211,18.814366,6657.455182,0.401119,1668.319496,175.0,14.959948
4,40.440146,-80.0065,118.40,5,15211,18.814366,6657.455182,0.401119,1668.319496,175.0,14.932009


In [ ]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

X = ['density2019', 'wfh_emp', 'dist_to_cbd', 'estab_count']
y = 'pred_post_pct_change'

#scaler = StandardScaler()
#dfx = pd.DataFrame(scaler.fit_transform(df[X]), columns = X)
df_clean = df[X+[y]]
df_clean=df_clean.dropna()

X_run=df_clean[X]
y_run=df_clean[y]

result = sm.OLS(y_run, X_run).fit()

print(result.summary())

                                  OLS Regression Results                                 
Dep. Variable:     pred_post_pct_change   R-squared (uncentered):                   0.819
Model:                              OLS   Adj. R-squared (uncentered):              0.819
Method:                   Least Squares   F-statistic:                              9804.
Date:                  Mon, 30 May 2022   Prob (F-statistic):                        0.00
Time:                          02:21:55   Log-Likelihood:                         -22918.
No. Observations:                  8693   AIC:                                  4.584e+04
Df Residuals:                      8689   BIC:                                  4.587e+04
Df Model:                             4                                                  
Covariance Type:              nonrobust                                                  
                  coef    std err          t      P>|t|      [0.025      0.975]
--------------------

In [ ]:
np.sqrt(mean_squared_error(y_run, result.predict(X_run), squared=False))

1.8380636941675401